In [ ]:
import matplotlib.pyplot as plt
import os
import collections
from glob import glob
src_path = "DATA/TRAIN"
sub_class = os.listdir(src_path)
print(sub_class)

In [ ]:
import tensorflow as tf
tf.config.threading.set_intra_op_parallelism_threads(2)
tf.config.threading.set_inter_op_parallelism_threads(2)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
import scipy

In [ ]:
SRC_PATH_TRAIN = 'DATA/TRAIN'
SRC_PATH_TEST = 'DATA/TEST'

In [ ]:
batch_size = 8
train_generator = train_datagen.flow_from_directory(
    directory=SRC_PATH_TRAIN,
    target_size=(100, 100),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    subset='training',
    shuffle=True,
    seed=123
)
valid_generator = train_datagen.flow_from_directory(
    directory=SRC_PATH_TRAIN,
    target_size=(100, 100),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    subset='validation',
    shuffle=True,
    seed=123
)
test_generator = test_datagen.flow_from_directory(
    directory=SRC_PATH_TEST,
    target_size=(100, 100),
    color_mode="rgb",
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=123
)

In [ ]:
for images_batch, labels_batch in train_generator:
    print(images_batch.shape)
    print(labels_batch.shape)
    break

In [ ]:
num_classes = 8
with tf.device('/CPU:0'):
    model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu',input_shape=(100, 100, 3)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(num_classes)
    ])

In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
import os
checkpoint_path = "training_3/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
model = create_model()

In [ ]:
with tf.device('/CPU:0'):
    history = model.fit(
            train_generator,
            epochs=10,
            validation_data=valid_generator,
            use_multiprocessing=True,
            callbacks=[cp_callback])


In [ ]:
score = model.evaluate(valid_generator)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
predict=model.predict(test_generator)
# predict the class label
y_classes = predict.argmax(axis=-1)

In [ ]:
score = model.evaluate(test_generator)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
def calculate_average_accurency(path_to_data_dir, image_width, image_height):
    data_container = collections.defaultdict(list)
    path_to_prediction_candidates = Path(path_to_data_dir)
    image_count = len(list(path_to_prediction_candidates.glob('*.png')))
    print(f'found {image_count} images in path {path_to_data_dir}"')
    print(f'testing test set...')
    all_images = list(path_to_prediction_candidates.glob('*'))
    for count, path_to_image in enumerate(all_images):
        img = keras.preprocessing.image.load_img(
            path_to_image, target_size=(image_width, image_height)
        )
        img_array = keras.preprocessing.image.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0)
        predictions = model.predict(img_array)
        list_of_predictions = softmax(predictions[0])
        for label, accurency in zip(class_names, list_of_predictions):
            data_container[label].append(accurency)
    for key, value in data_container.items():
        print(f'Average prediction accurency: {key} -> {str(round(np.mean(value),5))}')

In [ ]:
calculate_average_accurency('DATA/TEST_PRED/movie1_1440p/', 100, 100)

In [ ]:
calculate_average_accurency('DATA/TEST_PRED/movie2_1440p/', 100, 100)

In [ ]:
calculate_average_accurency('DATA/TEST_PRED/movie1_2160p/', 100, 100)

In [ ]:
calculate_average_accurency('DATA/TEST_PRED/movie2_2160p/', 100, 100)